In [32]:
!pip install polars 
!pip install pandas

import polars as pl
import json
import os
import datetime
import pandas as pd

In [43]:
MeetingDuration = timedelta(days = 0, hours = 2 )
#possible meeting times
start_datetime = datetime.datetime(2025,10,12,5,30)
end_datetime = datetime.datetime(2025,10,15,5,30)


attendees={
        "Mumbai": 2,
        "Shanghai": 3,
        "Hong Kong": 1,
        "Singapore": 2,
        "Sydney": 2
        }

2:00:00


In [36]:
# Read the CSV file
'''
a = 5

# Open and read the JSON file
def file_list(json_file):
    # Read the JSON file
    with open(json_file, "r") as file:
        data = json.load(file)
    start_date_str = data["availability_window"]["start"]
    end_date_str = data["availability_window"]["end"]

    start_date = datetime.datetime.strptime(start_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")
    end_date = datetime.datetime.strptime(end_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")

    files = []


    # Loop from start to end date within availability window
    current_date = start_date - datetime.timedelta(days=a)
    while current_date.date() <= end_date.date():
        # For each file, construct path like ChallengeData/schedules/2025/12/10.csv
        file_path = os.path.join(
            "schedules",
            str(current_date.year),
            f"{current_date.month:02d}",
            f"{current_date.day:02d}.csv"
        )
        files.append(file_path)
        current_date += datetime.timedelta(days=1)

    return files

# example path: /Users/ankithrangan/Desktop/ChallengeData/schedules/2023/05/04.csv
#/Users/ankithrangan/Desktop/ChallengeData/schedules/2025/03/03.csv
schedule_files=[file_list(json_file)] #list of csv files
new_files = schedule_files[0]
new_files
'''
#data cleaned elsewhere because i only have 1 day.

'\na = 5\n\n# Open and read the JSON file\ndef file_list(json_file):\n    # Read the JSON file\n    with open(json_file, "r") as file:\n        data = json.load(file)\n    start_date_str = data["availability_window"]["start"]\n    end_date_str = data["availability_window"]["end"]\n\n    start_date = datetime.datetime.strptime(start_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")\n    end_date = datetime.datetime.strptime(end_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")\n\n    files = []\n\n\n    # Loop from start to end date within availability window\n    current_date = start_date - datetime.timedelta(days=a)\n    while current_date.date() <= end_date.date():\n        # For each file, construct path like ChallengeData/schedules/2025/12/10.csv\n        file_path = os.path.join(\n            "schedules",\n            str(current_date.year),\n            f"{current_date.month:02d}",\n            f"{current_date.day:02d}.csv"\n        )\n        files.append(file_path)\n        current

In [ ]:
'''
for i in range(len(new_files)):
    new_files[i] = f"/Users/ankithrangan/Desktop/ChallengeData/{new_files[i]}"
new_files
'''

In [12]:
"""
from pathlib import Path
# If your file_list() returned relative paths like "schedules/2025/12/05.csv"
base = Path("/Users/ankithrangan/Desktop/ChallengeData")

# If schedule_files was [[...]], flatten it first:
flat_files = schedule_files[0] if isinstance(schedule_files[0], list) else schedule_files

# Build absolute paths and check existence
abs_paths = [base / Path(p.strip()) for p in flat_files]
existing = [str(p) for p in abs_paths if p.is_file()]
missing  = [str(p) for p in abs_paths if not p.is_file()]

if missing:
    print(f"Missing {len(missing)} files. First few:\n", "\n".join(missing[:5]))

if not existing:
    raise FileNotFoundError("No existing CSVs to read after filtering.")

# Combine them into a single DataFrame
schedules = pl.scan_csv(existing).collect()

"""


schedules=pl.scan_csv(new_files, infer_schema_length=10000).collect() #list of paths to access csv files (within availability)
schedules

#/Users/ankithrangan/Desktop/ChallengeData/schedules/2023/05/04.csv

NameError: name 'new_files' is not defined

In [13]:


airport_dictionary = {
    'Mumbai': ['BOM'],
    'London': ['LHR', 'STN', 'LGW', 'LTN'],
    'Paris': ['CDG', 'BVA', 'ORY'],
    'Hong Kong': ['HKG'],
    'Singapore': ['SIN'],
    'Shanghai': ['PVG', 'SHA'],
    'Dubai': ['DXB'],
    'Zürich': ['ZRH'],
    'Geneva': ['GVA'],
    'Aarhus': ['AAR'],
    'Sydney': ['SYD'],
    'Wroclaw': ['WRO'],
    'Budapest': ['BUD']
}

#adds corresponding country code to 'target codes'
target_codes = set()
for city in attendees.keys():
    if city in airport_dictionary:
        target_codes.update(airport_dictionary[city])

#filter relevant airports (ie in target_codes)

filter_expression = (
    (pl.col("DEPCITY").is_in(target_codes)) | 
    (pl.col("ARRCITY").is_in(target_codes))
)

filtered_schedules = schedules.filter(filter_expression) #gives all rows with airports mentioned in the input
filtered_schedules


NameError: name 'attendees' is not defined

In [ ]:
g

In [ ]:
gggggggggggggggggggggggggggggggg

NameError: name 'filtered_schedules' is not defined

In [11]:
filtered_schedules= filtered_schedules.with_columns(
    FLIGHT_DURATION = (
        pl.col("SCHEDULED_ARRIVAL_DATE_TIME_UTC").str.to_datetime() -
        pl.col("SCHEDULED_DEPARTURE_DATE_TIME_UTC").str.to_datetime()
    )
)

# 2. Select only the columns you want
final_df = filtered_schedules.select(
    "DEPCITY",
    "ARRCITY",
    "SCHEDULED_DEPARTURE_DATE_TIME_LOCAL",
    "SCHEDULED_ARRIVAL_DATE_TIME_LOCAL",
    "FLIGHT_DURATION"  # The new column we just made
)

# Display the final result
print(final_df.unique())

shape: (8_834, 5)
┌─────────┬─────────┬──────────────────────────────┬─────────────────────────────┬─────────────────┐
│ DEPCITY ┆ ARRCITY ┆ SCHEDULED_DEPARTURE_DATE_TIM ┆ SCHEDULED_ARRIVAL_DATE_TIME ┆ FLIGHT_DURATION │
│ ---     ┆ ---     ┆ E_…                          ┆ _LO…                        ┆ ---             │
│ str     ┆ str     ┆ ---                          ┆ ---                         ┆ duration[μs]    │
│         ┆         ┆ str                          ┆ str                         ┆                 │
╞═════════╪═════════╪══════════════════════════════╪═════════════════════════════╪═════════════════╡
│ ZHF     ┆ GVA     ┆ 2025-08-08 16:27:00.000      ┆ 2025-08-08 18:15:00.000     ┆ 1h 48m          │
│ GVA     ┆ QLS     ┆ 2025-08-01 14:25:00.000      ┆ 2025-08-01 15:14:00.000     ┆ 49m             │
│ ZKZ     ┆ GVA     ┆ 2025-08-01 08:30:00.000      ┆ 2025-08-01 09:44:00.000     ┆ 1h 14m          │
│ QLS     ┆ GVA     ┆ 2025-07-31 11:23:00.000      ┆ 2025-07-31 12:15:00.

In [12]:
emissions2 = pl.read_csv("/Users/ankithrangan/Desktop/ChallengeData/emissions.csv")
emissions2

SCHEDULED_FLIGHT_LEG_PK,CARRIER_CODE,OUTBOUND_CODE,SERVICE_SUFFIX,FLIGHT_NUMBER,DEPARTURE_AIRPORT,ARRIVAL_AIRPORT,SCHEDULED_DEPARTURE_DATE,AIRCRAFT_TYPE,SEATS,ESTIMATED_FUEL_BURN_TAXI_OUT_TONNES,ESTIMATED_FUEL_BURN_TAKEOFF_TONNES,ESTIMATED_FUEL_BURN_CLIMBOUT_TONNES,ESTIMATED_FUEL_BURN_CRUISE_TONNES,ESTIMATED_FUEL_BURN_APPROACH_TONNES,ESTIMATED_FUEL_BURN_TAXI_IN_TONNES,ESTIMATED_FUEL_BURN_TOTAL_TONNES,ESTIMATED_CO2_TAXI_OUT_TONNES,ESTIMATED_CO2_TAKEOFF_TONNES,ESTIMATED_CO2_CLIMBOUT_TONNES,ESTIMATED_CO2_CRUISE_TONNES,ESTIMATED_CO2_APPROACH_TONNES,ESTIMATED_CO2_TAXI_IN_TONNES,ESTIMATED_CO2_TOTAL_TONNES,MISSING_REFERENCE_FLIGHT_TIMES,LOAD_DATE,IS_OPERATING
str,str,str,str,i64,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,str,bool
"""921991ff51c02a70ff6c09cfb69425…","""0B""",null,null,761,"""CGK""","""SUB""","""2024-12-22""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,"""2024-12-05""",true
"""9a8f33c59f1cd15beaa23ad164178d…","""0B""",null,null,762,"""SUB""","""CGK""","""2024-12-08""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,"""2024-12-05""",true
"""3da312f362a107e999f0de6632e647…","""0B""",null,null,763,"""CGK""","""SUB""","""2024-12-30""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,"""2024-12-05""",true
"""263319d49997ab511ab864b7cd0126…","""0B""",null,null,764,"""SUB""","""CGK""","""2024-12-09""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,"""2024-12-05""",true
"""d4b79b0fa0f7d633e70c0bd9bdb35e…","""0B""",null,null,771,"""CGK""","""DPS""","""2024-12-21""","""738""",180,0.212,0.098,0.248,3.425,0.157,0.066,4.207,0.67,0.31,0.784,10.824,0.497,0.21,13.294,false,"""2024-12-05""",true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""f49b59b6577b132f086cb68c1cd67a…","""ZV""",null,null,420,"""CZM""","""CUN""","""2024-12-22""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,"""2024-12-05""",true
"""ac75acf7e1b5151c910635a3493246…","""ZV""",null,null,450,"""CTM""","""CUN""","""2024-12-13""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,"""2024-12-05""",true
"""07977f04339f9b546317ae1d459ec9…","""ZV""",null,null,451,"""CUN""","""CTM""","""2024-12-31""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,"""2024-12-05""",true


In [13]:
#eliminate flight code column
emissions2 = emissions2.drop("SCHEDULED_FLIGHT_LEG_PK", "CARRIER_CODE", "OUTBOUND_CODE", "SERVICE_SUFFIX", "LOAD_DATE", "SCHEDULED_DEPARTURE_DATE")
emissions2


FLIGHT_NUMBER,DEPARTURE_AIRPORT,ARRIVAL_AIRPORT,AIRCRAFT_TYPE,SEATS,ESTIMATED_FUEL_BURN_TAXI_OUT_TONNES,ESTIMATED_FUEL_BURN_TAKEOFF_TONNES,ESTIMATED_FUEL_BURN_CLIMBOUT_TONNES,ESTIMATED_FUEL_BURN_CRUISE_TONNES,ESTIMATED_FUEL_BURN_APPROACH_TONNES,ESTIMATED_FUEL_BURN_TAXI_IN_TONNES,ESTIMATED_FUEL_BURN_TOTAL_TONNES,ESTIMATED_CO2_TAXI_OUT_TONNES,ESTIMATED_CO2_TAKEOFF_TONNES,ESTIMATED_CO2_CLIMBOUT_TONNES,ESTIMATED_CO2_CRUISE_TONNES,ESTIMATED_CO2_APPROACH_TONNES,ESTIMATED_CO2_TAXI_IN_TONNES,ESTIMATED_CO2_TOTAL_TONNES,MISSING_REFERENCE_FLIGHT_TIMES,IS_OPERATING
i64,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool
761,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
762,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
763,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
764,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
771,"""CGK""","""DPS""","""738""",180,0.212,0.098,0.248,3.425,0.157,0.066,4.207,0.67,0.31,0.784,10.824,0.497,0.21,13.294,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
420,"""CZM""","""CUN""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,true
450,"""CTM""","""CUN""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,true
451,"""CUN""","""CTM""","""CNF""",12,null,null,null,null,null,null,null,null,null,null,null,null,null,null,true,true


In [14]:
emissions_nu = emissions2.drop_nulls()
emissions_nu

FLIGHT_NUMBER,DEPARTURE_AIRPORT,ARRIVAL_AIRPORT,AIRCRAFT_TYPE,SEATS,ESTIMATED_FUEL_BURN_TAXI_OUT_TONNES,ESTIMATED_FUEL_BURN_TAKEOFF_TONNES,ESTIMATED_FUEL_BURN_CLIMBOUT_TONNES,ESTIMATED_FUEL_BURN_CRUISE_TONNES,ESTIMATED_FUEL_BURN_APPROACH_TONNES,ESTIMATED_FUEL_BURN_TAXI_IN_TONNES,ESTIMATED_FUEL_BURN_TOTAL_TONNES,ESTIMATED_CO2_TAXI_OUT_TONNES,ESTIMATED_CO2_TAKEOFF_TONNES,ESTIMATED_CO2_CLIMBOUT_TONNES,ESTIMATED_CO2_CRUISE_TONNES,ESTIMATED_CO2_APPROACH_TONNES,ESTIMATED_CO2_TAXI_IN_TONNES,ESTIMATED_CO2_TOTAL_TONNES,MISSING_REFERENCE_FLIGHT_TIMES,IS_OPERATING
i64,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool
761,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
762,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
763,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
764,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
771,"""CGK""","""DPS""","""738""",180,0.212,0.098,0.248,3.425,0.157,0.066,4.207,0.67,0.31,0.784,10.824,0.497,0.21,13.294,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2330,"""KAN""","""ABV""","""735""",112,0.134,0.097,0.252,0.965,0.161,0.104,1.713,0.423,0.307,0.796,3.049,0.509,0.329,5.413,false,true
2332,"""LOS""","""ABV""","""735""",112,0.089,0.089,0.089,1.144,0.089,0.089,1.589,0.281,0.281,0.281,3.616,0.281,0.281,5.023,true,true
2333,"""ABV""","""LOS""","""733""",124,0.088,0.088,0.088,0.666,0.088,0.088,1.108,0.279,0.279,0.279,2.105,0.279,0.279,3.501,true,true


In [15]:
emissions_nu['MISSING_REFERENCE_FLIGHT_TIMES'].value_counts()

MISSING_REFERENCE_FLIGHT_TIMES,count
bool,u32
false,243736
true,39396


In [16]:
#drop 'missing reference flight times' column, as only dropping around 1/8 of the data and the data is unreliable
emissions_nu1 = emissions_nu.filter(pl.col("MISSING_REFERENCE_FLIGHT_TIMES") == "false")
emissions_nu1

FLIGHT_NUMBER,DEPARTURE_AIRPORT,ARRIVAL_AIRPORT,AIRCRAFT_TYPE,SEATS,ESTIMATED_FUEL_BURN_TAXI_OUT_TONNES,ESTIMATED_FUEL_BURN_TAKEOFF_TONNES,ESTIMATED_FUEL_BURN_CLIMBOUT_TONNES,ESTIMATED_FUEL_BURN_CRUISE_TONNES,ESTIMATED_FUEL_BURN_APPROACH_TONNES,ESTIMATED_FUEL_BURN_TAXI_IN_TONNES,ESTIMATED_FUEL_BURN_TOTAL_TONNES,ESTIMATED_CO2_TAXI_OUT_TONNES,ESTIMATED_CO2_TAKEOFF_TONNES,ESTIMATED_CO2_CLIMBOUT_TONNES,ESTIMATED_CO2_CRUISE_TONNES,ESTIMATED_CO2_APPROACH_TONNES,ESTIMATED_CO2_TAXI_IN_TONNES,ESTIMATED_CO2_TOTAL_TONNES,MISSING_REFERENCE_FLIGHT_TIMES,IS_OPERATING
i64,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool
761,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
762,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
763,"""CGK""","""SUB""","""738""",180,0.225,0.098,0.248,2.325,0.157,0.066,3.119,0.711,0.31,0.784,7.346,0.497,0.21,9.857,false,true
764,"""SUB""","""CGK""","""738""",180,0.176,0.098,0.248,2.673,0.157,0.11,3.462,0.558,0.31,0.784,8.446,0.497,0.346,10.941,false,true
771,"""CGK""","""DPS""","""738""",180,0.212,0.098,0.248,3.425,0.157,0.066,4.207,0.67,0.31,0.784,10.824,0.497,0.21,13.294,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
306,"""JNB""","""LUN""","""73W""",118,0.175,0.092,0.239,3.34,0.153,0.081,4.08,0.554,0.291,0.754,10.554,0.484,0.257,12.893,false,true
307,"""LUN""","""JNB""","""73W""",118,0.136,0.095,0.244,3.766,0.156,0.095,4.492,0.43,0.301,0.772,11.901,0.492,0.3,14.197,false,true
524,"""DAR""","""NBO""","""73W""",118,0.177,0.099,0.252,1.907,0.158,0.123,2.717,0.56,0.313,0.797,6.025,0.502,0.389,8.586,false,true


In [17]:
emissions_nu2 = emissions_nu1.select('DEPARTURE_AIRPORT','ARRIVAL_AIRPORT','AIRCRAFT_TYPE','ESTIMATED_CO2_TOTAL_TONNES', 'SEATS').unique()
emissions_nu2

DEPARTURE_AIRPORT,ARRIVAL_AIRPORT,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS
str,str,str,f64,i64
"""KUL""","""AUH""","""789""",105.01,290
"""DXB""","""BAH""","""77L""",29.967,302
"""FRA""","""BER""","""320""",7.4,168
"""KMG""","""WUH""","""736""",14.964,181
"""LUX""","""TFS""","""73W""",32.511,141
…,…,…,…,…
"""PVG""","""ICN""","""333""",32.906,298
"""DUB""","""ZRH""","""223""",9.327,145
"""DTM""","""STN""","""73H""",8.141,189


In [18]:
airport_dictionary = {

    'Mumbai': ['BOM'],

    'London': ['LHR', 'LGW', 'LTN', 'STN'],

    'Paris': ['CDG', 'ORY', 'BVA'],

    'Hong Kong': ['HKG'],

    'Singapore': ['SIN'],

    'Shanghai': ['PVG', 'SHA'],

    'Dubai': ['DXB'],

    'Zürich': ['ZRH'],

    'Geneva': ['GVA'],

    'Aarhus': ['AAR'],

    'Sydney': ['SYD'],

    'Wroclaw': ['WRO'],

    'Budapest': ['BUD']

}

 

 
'''
target_codes = set()

    if city in airport_dictionary:

        target_codes.update(airport_dictionary[city])

'''
target_codes=set()

for city in airport_dictionary:
    target_codes.update(airport_dictionary[city])

filter_expression = (

    (pl.col("DEPARTURE_AIRPORT").is_in(target_codes)) |

    (pl.col("ARRIVAL_AIRPORT").is_in(target_codes))

)



In [19]:
filtered_emissions = emissions_nu2.filter(filter_expression)
matching_emissions=filtered_emissions.rename({'DEPARTURE_AIRPORT':'DEPCITY','ARRIVAL_AIRPORT':'ARRCITY'})
matching_emissions

DEPCITY,ARRCITY,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS
str,str,str,f64,i64
"""DXB""","""BAH""","""77L""",29.967,302
"""CDG""","""ALG""","""73H""",17.415,148
"""PEK""","""HKG""","""32Q""",22.641,202
"""XIY""","""PVG""","""319""",15.925,122
"""AGA""","""LGW""","""319""",24.046,156
…,…,…,…,…
"""SHA""","""PEK""","""332""",37.75,214
"""LHW""","""PVG""","""320""",20.17,158
"""PVG""","""ICN""","""333""",32.906,298


In [20]:
#print(matching_emissions.group_by(['DEPCITY','ARRCITY']).count().filter(pl.col('count')>1))
emissions_lookup=matching_emissions.unique(subset=['DEPCITY','ARRCITY'],keep='first')
emissions_lookup

DEPCITY,ARRCITY,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS
str,str,str,f64,i64
"""LHR""","""TFU""","""333""",193.103,301
"""BVA""","""PSA""","""73H""",10.966,189
"""SHA""","""LYA""","""320""",14.413,158
"""BUD""","""FRA""","""32Q""",9.637,215
"""ZRH""","""PMI""","""32B""",14.635,219
…,…,…,…,…
"""SIN""","""PER""","""359""",75.453,303
"""GVA""","""MAD""","""320""",13.15,180
"""BUD""","""CPH""","""73H""",11.311,189


In [21]:
merge_df=final_df.join(emissions_lookup,on=['DEPCITY','ARRCITY'],how='left').unique().drop_nulls()


In [22]:
merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS
str,str,str,str,duration[μs],str,f64,i64
"""ZRH""","""GVA""","""2025-08-01 14:00:00.000""","""2025-08-01 19:00:00.000""",5h,"""221""",3.174,125
"""WAW""","""GVA""","""2025-08-03 16:35:00.000""","""2025-08-03 19:00:00.000""",2h 25m,"""7M8""",12.607,189
"""ZRH""","""GVA""","""2025-08-07 09:08:00.000""","""2025-08-07 12:25:00.000""",3h 17m,"""221""",3.174,125
"""GVA""","""OPO""","""2025-08-08 16:05:00.000""","""2025-08-08 17:30:00.000""",2h 25m,"""E95""",12.154,118
"""BOD""","""GVA""","""2025-08-04 16:50:00.000""","""2025-08-04 18:10:00.000""",1h 20m,"""320""",8.202,186
…,…,…,…,…,…,…,…
"""MAD""","""GVA""","""2025-08-06 09:50:00.000""","""2025-08-06 11:45:00.000""",1h 55m,"""319""",11.452,156
"""TUN""","""GVA""","""2025-08-06 15:15:00.000""","""2025-08-06 18:10:00.000""",1h 55m,"""320""",11.831,162
"""GVA""","""ZRH""","""2025-08-08 07:25:00.000""","""2025-08-08 08:15:00.000""",50m,"""223""",3.316,145


In [23]:

merge_df = merge_df.with_columns(
    pl.col("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL")
    .str.to_datetime(strict=False),
    pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL")
    .str.to_datetime(strict=False)
)
merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS
str,str,datetime[μs],datetime[μs],duration[μs],str,f64,i64
"""ZRH""","""GVA""",2025-08-01 14:00:00,2025-08-01 19:00:00,5h,"""221""",3.174,125
"""WAW""","""GVA""",2025-08-03 16:35:00,2025-08-03 19:00:00,2h 25m,"""7M8""",12.607,189
"""ZRH""","""GVA""",2025-08-07 09:08:00,2025-08-07 12:25:00,3h 17m,"""221""",3.174,125
"""GVA""","""OPO""",2025-08-08 16:05:00,2025-08-08 17:30:00,2h 25m,"""E95""",12.154,118
"""BOD""","""GVA""",2025-08-04 16:50:00,2025-08-04 18:10:00,1h 20m,"""320""",8.202,186
…,…,…,…,…,…,…,…
"""MAD""","""GVA""",2025-08-06 09:50:00,2025-08-06 11:45:00,1h 55m,"""319""",11.452,156
"""TUN""","""GVA""",2025-08-06 15:15:00,2025-08-06 18:10:00,1h 55m,"""320""",11.831,162
"""GVA""","""ZRH""",2025-08-08 07:25:00,2025-08-08 08:15:00,50m,"""223""",3.316,145


In [24]:
merge_df = merge_df.with_columns(
    (pl.col("ESTIMATED_CO2_TOTAL_TONNES") / pl.col("SEATS")).alias("EMISSION_PER_SEAT")
)
merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS,EMISSION_PER_SEAT
str,str,datetime[μs],datetime[μs],duration[μs],str,f64,i64,f64
"""ZRH""","""GVA""",2025-08-01 14:00:00,2025-08-01 19:00:00,5h,"""221""",3.174,125,0.025392
"""WAW""","""GVA""",2025-08-03 16:35:00,2025-08-03 19:00:00,2h 25m,"""7M8""",12.607,189,0.066704
"""ZRH""","""GVA""",2025-08-07 09:08:00,2025-08-07 12:25:00,3h 17m,"""221""",3.174,125,0.025392
"""GVA""","""OPO""",2025-08-08 16:05:00,2025-08-08 17:30:00,2h 25m,"""E95""",12.154,118,0.103
"""BOD""","""GVA""",2025-08-04 16:50:00,2025-08-04 18:10:00,1h 20m,"""320""",8.202,186,0.044097
…,…,…,…,…,…,…,…,…
"""MAD""","""GVA""",2025-08-06 09:50:00,2025-08-06 11:45:00,1h 55m,"""319""",11.452,156,0.07341
"""TUN""","""GVA""",2025-08-06 15:15:00,2025-08-06 18:10:00,1h 55m,"""320""",11.831,162,0.073031
"""GVA""","""ZRH""",2025-08-08 07:25:00,2025-08-08 08:15:00,50m,"""223""",3.316,145,0.022869


In [42]:
def departure_time(data,city0,city1,arrive_before_time):

    # 1. Calculate the arrival time for all flights # and filter out flights that don't match our criteria. 
    best_dept_flight = data.with_columns((pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") + pl.col("FLIGHT_DURATION")).alias("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL")).filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY") == city1) & (pl.col("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL") < arrive_before_time)).sort("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL", descending=True ).head(1) # 3. Select the top one return best_flight
    #relevant_data = data.filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY")) == city1)& (arrive_before_time >pl.col('SCHEDULED_ARRIVAL_DATE_TIME_LOCAL'))
    
    return best_dept_flight['SCHEDULED_DEPARTURE_DATE_TIME_LOCAL','EMISSION_PER_SEAT']

departure_time(merge_df,'GVA','OPO',datetime.datetime(2025,12,11,11,0,0))

InvalidOperationError: cannot compare 'date/datetime/time' to a string value (create native python { 'date', 'datetime', 'time' } or compare to a temporal column)

In [26]:
def arrival_time(data,city0,city1,meeting_end):

    # 1. Calculate the departure time for all flights # and filter out flights that don't match our criteria. 
    best_arv_flight = data.with_columns((pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") + pl.col("FLIGHT_DURATION")).alias("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL")).filter((pl.col("DEPCITY") == city1) & (pl.col("ARRCITY") == city0) & (pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") > meeting_end)).sort("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL", descending=False).head(1) # 3. Select the top one return best_flight
    #relevant_data = data.filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY")) == city1)& (arrive_before_time >pl.col('SCHEDULED_ARRIVAL_DATE_TIME_LOCAL'))
    
    return best_arv_flight['SCHEDULED_ARRIVAL_DATE_TIME_LOCAL','EMISSION_PER_SEAT']

arrival_time(merge_df,'SIN','HKG',datetime.datetime(2025,12,12,14,0,0))

SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,EMISSION_PER_SEAT
datetime[μs],f64


In [27]:
type(merge_df)

polars.dataframe.frame.DataFrame

In [28]:
#converting datetime back to str so can be saved as csv file

merge_df = merge_df.with_columns([
    pl.col("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL").dt.strftime("%Y-%m-%d %H:%M:%S").alias("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL"),
    pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL").dt.strftime("%Y-%m-%d %H:%M:%S").alias("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL")
])
merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS,EMISSION_PER_SEAT
str,str,str,str,duration[μs],str,f64,i64,f64
"""ZRH""","""GVA""","""2025-08-01 14:00:00""","""2025-08-01 19:00:00""",5h,"""221""",3.174,125,0.025392
"""WAW""","""GVA""","""2025-08-03 16:35:00""","""2025-08-03 19:00:00""",2h 25m,"""7M8""",12.607,189,0.066704
"""ZRH""","""GVA""","""2025-08-07 09:08:00""","""2025-08-07 12:25:00""",3h 17m,"""221""",3.174,125,0.025392
"""GVA""","""OPO""","""2025-08-08 16:05:00""","""2025-08-08 17:30:00""",2h 25m,"""E95""",12.154,118,0.103
"""BOD""","""GVA""","""2025-08-04 16:50:00""","""2025-08-04 18:10:00""",1h 20m,"""320""",8.202,186,0.044097
…,…,…,…,…,…,…,…,…
"""MAD""","""GVA""","""2025-08-06 09:50:00""","""2025-08-06 11:45:00""",1h 55m,"""319""",11.452,156,0.07341
"""TUN""","""GVA""","""2025-08-06 15:15:00""","""2025-08-06 18:10:00""",1h 55m,"""320""",11.831,162,0.073031
"""GVA""","""ZRH""","""2025-08-08 07:25:00""","""2025-08-08 08:15:00""",50m,"""223""",3.316,145,0.022869


In [29]:


merge_df = merge_df.with_columns(
    pl.format(
        "{}h {}m",
        (pl.col("FLIGHT_DURATION").dt.total_minutes() // 60),
        (pl.col("FLIGHT_DURATION").dt.total_minutes() % 60),
    ).alias("FLIGHT_DURATION_STR")
)


merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS,EMISSION_PER_SEAT,FLIGHT_DURATION_STR
str,str,str,str,duration[μs],str,f64,i64,f64,str
"""ZRH""","""GVA""","""2025-08-01 14:00:00""","""2025-08-01 19:00:00""",5h,"""221""",3.174,125,0.025392,"""5h 0m"""
"""WAW""","""GVA""","""2025-08-03 16:35:00""","""2025-08-03 19:00:00""",2h 25m,"""7M8""",12.607,189,0.066704,"""2h 25m"""
"""ZRH""","""GVA""","""2025-08-07 09:08:00""","""2025-08-07 12:25:00""",3h 17m,"""221""",3.174,125,0.025392,"""3h 17m"""
"""GVA""","""OPO""","""2025-08-08 16:05:00""","""2025-08-08 17:30:00""",2h 25m,"""E95""",12.154,118,0.103,"""2h 25m"""
"""BOD""","""GVA""","""2025-08-04 16:50:00""","""2025-08-04 18:10:00""",1h 20m,"""320""",8.202,186,0.044097,"""1h 20m"""
…,…,…,…,…,…,…,…,…,…
"""MAD""","""GVA""","""2025-08-06 09:50:00""","""2025-08-06 11:45:00""",1h 55m,"""319""",11.452,156,0.07341,"""1h 55m"""
"""TUN""","""GVA""","""2025-08-06 15:15:00""","""2025-08-06 18:10:00""",1h 55m,"""320""",11.831,162,0.073031,"""1h 55m"""
"""GVA""","""ZRH""","""2025-08-08 07:25:00""","""2025-08-08 08:15:00""",50m,"""223""",3.316,145,0.022869,"""0h 50m"""


In [30]:
merge_df.select(["FLIGHT_DURATION", "FLIGHT_DURATION_STR"]).head()
print(merge_df.schema)  # FLIGHT_DURATION_STR -> Utf8


Schema({'DEPCITY': String, 'ARRCITY': String, 'SCHEDULED_DEPARTURE_DATE_TIME_LOCAL': String, 'SCHEDULED_ARRIVAL_DATE_TIME_LOCAL': String, 'FLIGHT_DURATION': Duration(time_unit='us'), 'AIRCRAFT_TYPE': String, 'ESTIMATED_CO2_TOTAL_TONNES': Float64, 'SEATS': Int64, 'EMISSION_PER_SEAT': Float64, 'FLIGHT_DURATION_STR': String})


In [31]:
merge_df = merge_df.with_columns(
    pl.format(
        "{}h {}m",
        (pl.col("FLIGHT_DURATION").dt.total_minutes() // 60),
        (pl.col("FLIGHT_DURATION").dt.total_minutes() % 60),
    ).alias("FLIGHT_DURATION")  # same name → replaces old one
)


In [32]:
merge_df

DEPCITY,ARRCITY,SCHEDULED_DEPARTURE_DATE_TIME_LOCAL,SCHEDULED_ARRIVAL_DATE_TIME_LOCAL,FLIGHT_DURATION,AIRCRAFT_TYPE,ESTIMATED_CO2_TOTAL_TONNES,SEATS,EMISSION_PER_SEAT,FLIGHT_DURATION_STR
str,str,str,str,str,str,f64,i64,f64,str
"""ZRH""","""GVA""","""2025-08-01 14:00:00""","""2025-08-01 19:00:00""","""5h 0m""","""221""",3.174,125,0.025392,"""5h 0m"""
"""WAW""","""GVA""","""2025-08-03 16:35:00""","""2025-08-03 19:00:00""","""2h 25m""","""7M8""",12.607,189,0.066704,"""2h 25m"""
"""ZRH""","""GVA""","""2025-08-07 09:08:00""","""2025-08-07 12:25:00""","""3h 17m""","""221""",3.174,125,0.025392,"""3h 17m"""
"""GVA""","""OPO""","""2025-08-08 16:05:00""","""2025-08-08 17:30:00""","""2h 25m""","""E95""",12.154,118,0.103,"""2h 25m"""
"""BOD""","""GVA""","""2025-08-04 16:50:00""","""2025-08-04 18:10:00""","""1h 20m""","""320""",8.202,186,0.044097,"""1h 20m"""
…,…,…,…,…,…,…,…,…,…
"""MAD""","""GVA""","""2025-08-06 09:50:00""","""2025-08-06 11:45:00""","""1h 55m""","""319""",11.452,156,0.07341,"""1h 55m"""
"""TUN""","""GVA""","""2025-08-06 15:15:00""","""2025-08-06 18:10:00""","""1h 55m""","""320""",11.831,162,0.073031,"""1h 55m"""
"""GVA""","""ZRH""","""2025-08-08 07:25:00""","""2025-08-08 08:15:00""","""0h 50m""","""223""",3.316,145,0.022869,"""0h 50m"""


In [33]:
from pathlib import Path

out_dir = Path("/Users/ankithrangan/Desktop/ChallengeData")
out_dir.mkdir(parents=True, exist_ok=True)  # create the folder if missing

merge_df.write_csv(out_dir /"merged_data.csv")


In [35]:
import json
import os
import polars as pl
import datetime 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

Days_Before = 5

json_file_easy = "/Users/ankithrangan/Desktop/ChallengeData/seen_files/input_1.json"
json_file_medium = "/Users/ankithrangan/Desktop/ChallengeData/seen_files/input_2.json"
json_file_hard = "/Users/ankithrangan/Desktop/ChallengeData/seen_files/input_3.json"

data = pl.read_csv("/Users/ankithrangan/Desktop/ChallengeData/merged_data.csv")

    # Open and read the JSON file
    def file_list(json_file):
        # Read the JSON file
        with open(json_file, "r") as file:
            data = json.load(file)
        start_date_str = data["availability_window"]["start"]
        end_date_str = data["availability_window"]["end"]
        
        start_date = datetime.datetime.strptime(start_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")
        end_date = datetime.datetime.strptime(end_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")
        
        files = []
    
        # Loop from start to end date
        current_date = start_date - timedelta(days=Days_Before)
        while current_date.date() <= end_date.date():
            # Construct path like data/2025/12/10.csv
            file_path = os.path.join(
                "data",     # replace with name of file
                str(current_date.year),
                f"{current_date.month:02d}",
                f"{current_date.day:02d}.csv"
            )
            files.append(file_path)
            current_date += timedelta(days=1)
            
        return files

     def Event_Dur(json_file):
        # Read the JSON file
        with open(json_file, "r") as file:
            data = json.load(file)
    
        duration_days = data["event_duration"].get("days", 0)
        duration_hours = data["event_duration"].get("hours", 0)
        duration_minutes = data["event_duration"].get("minutes", 0)
        duration_seconds = data["event_duration"].get("seconds", 0)
        time_delta_duration = timedelta(days = duration_days,
        hours = duration_hours,
        minutes = duration_minutes,
        seconds = duration_seconds
        )
    
    
        return time_delta_duration
    
  


IndentationError: unindent does not match any outer indentation level (<string>, line 45)

In [ ]:
  #---------------------------------------------------------------------------------------------------------------

    Event_Length = Event_Dur(json_file_easy)
    
    target_score = {}
    
    Weight_Vector = [1,1,1]
    
    targets = [#airports] 
    
    iterating_time = 1  
    
    Min_Length_Per_Day = timedelta(hours=1)
    
    Local_wdt_start = 09:00:00
    Local_wdt_end = 17:00:00

    schedule_files=[file_list(json_file)] #list of csv files
    new_files = schedule_files[0]
    new_files

In [ ]:
  #-----------------------------------------------------------------------------------------------------------------

    
    def find_score(data,city0,city1,arrive_before_time,meeting_end):
    
        # 1. Calculate the arrival time for all flights # and filter out flights that don't match our criteria. 
        best_dept_flight = data.with_columns((pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") + pl.col("FLIGHT_DURATION")).alias("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL")).filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY") == city1) & (pl.col("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL") < arrive_before_time)).sort("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL", descending=True ).head(1) # 3. Select the top one return best_flight
        #relevant_data = data.filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY")) == city1)& (arrive_before_time >pl.col('SCHEDULED_ARRIVAL_DATE_TIME_LOCAL'))
        
       # return best_dept_flight['SCHEDULED_DEPARTURE_DATE_TIME_LOCAL','ESTIMATED_CO2_TOTAL_TONNES']
    
    
    
    
    #def arrival_time(data,city0,city1,meeting_end):
    
        # 1. Calculate the departure time for all flights # and filter out flights that don't match our criteria. 
        best_arv_flight = data.with_columns((pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") + pl.col("FLIGHT_DURATION")).alias("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL")).filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY") == city1) & (pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL") > meeting_end)).sort("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL", descending=False).head(1) # 3. Select the top one return best_flight
        #relevant_data = data.filter((pl.col("DEPCITY") == city0) & (pl.col("ARRCITY")) == city1)& (arrive_before_time >pl.col('SCHEDULED_ARRIVAL_DATE_TIME_LOCAL'))
        
        #return best_arv_flight['SCHEDULED_ARRIVAL_DATE_TIME_LOCAL','ESTIMATED_CO2_TOTAL_TONNES']
    
    
    #def difference():
        dep_optimal = best_dept_flight['SCHEDULED_DEPARTURE_DATE_TIME_LOCAL','ESTIMATED_CO2_TOTAL_TONNES']#departure_time(data,city0,city1,arrive_before_time)
        arr_optimal = best_arv_flight['SCHEDULED_ARRIVAL_DATE_TIME_LOCAL','ESTIMATED_CO2_TOTAL_TONNES']#arrival_time(data,city0,city1,meeting_end)
        difference = dep_optimal['SCHEDULED_DEPARTURE_DATE_TIME_LOCAL'][0] - arrival_time['SCHEDULED_ARRIVAL_DATE_TIME_LOCAL'][0]
        #return difference
        velctor = [difference,dep_optimal[1]+arr_optimal[1]]
        return vector

In [ ]:
  
    def Make_DT(T, DT) :
        return datetime.combine(dateTimeInput.date(), timeInput.time())
        
    def End(start_time) :
        nightlength = (Make_DT(UTC_wdt_start, start_time) - Make_DT(UTC_wdt_end, start_time)) + timedelta(days=1) +
        daylength = Make_DT(UTC_wdt_end, start_time) - Make_DT(UTC_wdt_start, start_time)
        nights = (Event_Length//daylength)
        return start_time + Event_Length + (nightlength * nights)

In [ ]:
for target in targets :   #something to do with if dep and arr being the same gives time and emission as zero
    
    UTC_wdt_start = UTC - LOCAL + Local_wdt_start
    UTC_wdt_end = UTC - LOCAL + Local_wdt_end
    
    #initial start and end
    Start_Time = start_date   
    End_Time = End(Start_Time)
    
    Start_Times = []
    End_Times = []


#------------------ITERATE IN RANGE OF POSIBLE MEETING TIME----------------

    while End_Time <= end_date :
        good = true
        if End_Time < UTC_wdt_start + Min_Length_Per_Day :  
            good = false
        if Start_Time + Min_Length_Per_Day > Make_DT(UTC_wdt_end, Start_Time) or Start_Time < Make_DT(UTC_wdt_start, Start_Time) :
            good = false
        if good == true : #read iff start end time -> meeting time per day in range(min,max)
            #Start_Times.append(Start_Time)  
            #End_Times.append(Start_Time)

        #------------------ITERATE WRT ATTENDEES----------------
            times = []
            emissions = []
            
            for j in attendees :
                emissions.append(find_score(j,target)[1])
                times.append(find_score(j,target)[0])
                emissions.append(find_score(j,target)[0])
                times.append(find_score(j,target)[1])
    
            T_emission = sum(emissions)
            A_time = sum(times)/(len(times))
            fairness = np.std(times)
    
            score = T_emission * Weight_Vector[0] + A_time * Weight_Vector[1] + fairness * Weight_Vector[2]

#---------------------------------------------------------------------
        Start_Time += iterating_time #iterate         #times
        End_Time = End(Start_Time)   #next end

    
   # target_score[target] = [score, time]   #value for specific time   
